In [0]:
%pip install lightgbm

In [0]:
# Configuration - parameterized for reuse
storage_account = "stgm5forecastdev"

account_key = dbutils.secrets.get(scope="m5-scope", key="adls-key")

spark.conf.set(
    f"fs.azure.account.key.{storage_account}.dfs.core.windows.net",
    account_key
)

raw_path = f"abfss://raw@{storage_account}.dfs.core.windows.net"
curated_path = f"abfss://curated@{storage_account}.dfs.core.windows.net"

display(dbutils.fs.ls(raw_path))


In [ ]:
# Add src to path for imports (Databricks Repos)
import sys
import os

# Get the repo root (parent of notebooks/)
repo_root = os.path.dirname(os.path.abspath("."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

# Import ETL functions from src
from src.etl import transform_sales_to_long, write_parquet

## Smoke Test ETL Step

In [0]:
from pyspark.sql.functions import col

file_path = f"{raw_path}/sales_train_evaluation.csv"

df = (spark.read
      .option("header", "true")
      .csv(file_path))

# Keep id columns + first 7 days as integers for a smoke test
id_cols = ["id", "item_id", "dept_id", "store_id", "cat_id", "state_id"]
value_cols = [c for c in df.columns if c.startswith("d_")][:7]

df_small = df.select(
    *id_cols,
    *[col(c).cast("int").alias(c) for c in value_cols]
)

output_path = f"{curated_path}/sales_small/"

(df_small
 .write
 .mode("overwrite")
 .parquet(output_path))

display(dbutils.fs.ls(output_path))


## Creating a long-format table for CA1  

In [0]:
# Main ETL: Transform sales data to long format using src/etl module
# This replaces inline transformation logic with reusable functions

STORE_ID = "CA_1"

# Transform: read raw -> filter store -> wide to long -> add day_num -> join calendar
df_long = transform_sales_to_long(
    spark=spark,
    raw_path=raw_path,
    store_id=STORE_ID,
    add_day_num=True,
    calendar_path=raw_path,
)

# Write to curated zone
output_path = f"{curated_path}/m5_daily_{STORE_ID.lower()}/"
write_parquet(
    df=df_long,
    output_path=output_path,
    partition_by=["store_id", "item_id"],
    mode="overwrite"
)

display(dbutils.fs.ls(output_path))


## Sanity-checking the long-table format for CA1

In [0]:
# Read back curated data for sanity check
daily_path = f"{curated_path}/m5_daily_{STORE_ID.lower()}/"
df_daily = spark.read.parquet(daily_path)

df_daily.printSchema()
df_daily.show(5)
print(f"Total rows: {df_daily.count():,}")


In [0]:
# day_num is now included from ETL - just verify
df_daily.select("id", "store_id", "item_id", "d", "day_num", "sales").show(5)


In [0]:
from pyspark.sql.functions import col

item_example = df_daily.select("item_id").first()["item_id"]
print("Using item:", item_example)

df_item = (df_daily
           .filter(col("item_id") == item_example)
           .orderBy("day_num"))

pdf = df_item.select("day_num", "sales").toPandas()
pdf.head()

In [0]:
import lightgbm as lgb
from src.model import smape  # Use SMAPE from src module

pdf = pdf.dropna().sort_values("day_num")

horizon = 28  # last 28 days as validation
train = pdf.iloc[:-horizon]
val = pdf.iloc[-horizon:]

X_train = train[["day_num"]]
y_train = train["sales"]
X_val = val[["day_num"]]
y_val = val["sales"]

train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

params = {
    "objective": "regression",
    "metric": "mae",
    "verbosity": -1,
}

model = lgb.train(
    params,
    train_data,
    num_boost_round=200,
    valid_sets=[val_data],
    callbacks=[lgb.early_stopping(stopping_rounds=20, verbose=False)],
)

y_pred = model.predict(X_val, num_iteration=model.best_iteration)
print("SMAPE:", smape(y_val, y_pred))

